In [47]:
# 【1】获取超级鹰识别的坐标：文档代码: https://www.chaojiying.com/user/mysoft/
import requests
from hashlib import md5
class Chaojiying_Client(object):
    def __init__(self, username, password, soft_id):
        self.username = username
        password =  password.encode('utf8')
        self.password = md5(password).hexdigest()
        self.soft_id = soft_id
        self.base_params = {'user': self.username,'pass2': self.password, 'softid': self.soft_id,}
        self.headers = {'Connection': 'Keep-Alive',
            'User-Agent': 'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0)', }
    def PostPic(self, im, codetype):
        """im: 图片字节,codetype: 题目类型 参考 http://www.chaojiying.com/price.html"""
        params = { 'codetype': codetype,}
        params.update(self.base_params)
        files = {'userfile': ('ccc.jpg', im)}
        r = requests.post('http://upload.chaojiying.net/Upload/Processing.php', data=params, files=files, headers=self.headers)
        return r.json()
    def ReportError(self, im_id):
        """ im_id:报错题目的图片ID"""
        params = {'id': im_id,}
        params.update(self.base_params)
        r = requests.post('http://upload.chaojiying.net/Upload/ReportError.php', data=params, headers=self.headers)
        return r.json()

def submit_img():       # 将验证码提交给超级鹰进行识别
    chaojiying = Chaojiying_Client('18820147698', 'cfy122333..', '915194')  # 用户中心>>软件ID 生成一个
    im = open(r'C:\Users\hp\Desktop\small_img.png', 'rb').read()            # 本地图片文件路径 来替换 a.jpg 有时WIN系统须要//
    result = chaojiying.PostPic(im, 9004)['pic_str']
    print(result)
    return result
submit_img()

149,38|73,45|37,75


'149,38|73,45|37,75'

In [49]:
# 【1】处理超级鹰返回的坐标：
result="149,38|73,45|37,75"
def parse_data(result):
    all_list = []       # 存储即将被点击的点的坐标  [[x1,y1],[x2,y2]]
    if '|' in result:
        list_1 = result.split('|')
        count_1 = len(list_1)
        for i in range(count_1):
            xy_list = []
            x = int(list_1[i].split(',')[0])
            y = int(list_1[i].split(',')[1])
            xy_list.append(x)
            xy_list.append(y)
            all_list.append(xy_list)
    else:
        print(result.split(',')[0])
        print(result.split(',')[1])

        x = int(result.split(',')[0])
        y = int(result.split(',')[1])
        xy_list = []
        xy_list.append(x)
        xy_list.append(y)
        all_list.append(xy_list)
    print(all_list)
    return all_list
parse_data(result)

[[149, 38], [73, 45], [37, 75]]


[[149, 38], [73, 45], [37, 75]]

In [52]:
# 【1】selenium登陆站 + 超级鹰——登陆B站：

from selenium import webdriver
from time import sleep
from PIL import Image
from selenium.webdriver import ActionChains
import random
img=[]
bro = webdriver.Chrome('./chromedriver.exe')                   # 使用selenium打开登录界面
def login():
    bro.get('https://passport.bilibili.com/login')
    bro.maximize_window()
    username = bro.find_element_by_id('login-username')      # 定位用户名，密码输入框
    password = bro.find_element_by_id('login-passwd')
    sleep(1)
    username.send_keys('18820147698')
    sleep(1)
    password.send_keys('cfy122333..')
    sleep(1)
    bro.find_element_by_xpath('//*[@id="geetest-wrap"]/div/div[5]/a[1]').click()       # 定位登录按钮并点击
    sleep(random.random()*3)
    sleep(3)
def save_img():                             
    bro.save_screenshot(r'C:\Users\hp\Desktop\page.png')         # save_screenshot就是将当前页面进行截图且保
    code_img_ele = bro.find_element_by_xpath('/html/body/div[3]/div[2]/div[6]')  # 定位验证码图片的位置
    location = code_img_ele.location        # 验证码左上角的坐标x,y
    size = code_img_ele.size                # 验证码图片对应的长和宽
    print('验证码位置：',location,'验证码size：',size)
    img.append(location)
    img.append(size)
    # 左上角和右下角的坐标,是否需要*1.24，在于电脑屏幕的比例是不是100%，请自行在设置中查看
    rangle = (
        int(location['x'] ), int(location['y'] ), int((location['x'] + size['width']) ),
        int((location['y'] + size['height']) )
    )
    i = Image.open(r'C:\Users\hp\Desktop\page.png')
    code_img_name = r'C:\Users\hp\Desktop\code.png'
    frame = i.crop(rangle)                  # crop根据rangle元组内的坐标进行裁剪
    frame.save(code_img_name)
    return code_img_ele

# 因为超级鹰识别的验证码图片的宽和高有限制，最好不超过460px*310px。
# 但是截取到的验证码图片宽高为338px*432px，这时就要先将图片缩小一倍再提交即可，等到收到坐标数据再将坐标乘2。
def narrow_img():       # 缩小图片
    code = Image.open(r'C:\Users\hp\Desktop\code.png')
    small_img = code.resize((int(round(img[1]['height']/2,0)),int(round((img[1]['width'])/2,0))))
    small_img.save(r'C:\Users\hp\Desktop\small_img.png')
# ==================================================================================
import requests
from hashlib import md5
class Chaojiying_Client(object):
    def __init__(self, username, password, soft_id):
        self.username = username
        password =  password.encode('utf8')
        self.password = md5(password).hexdigest()
        self.soft_id = soft_id
        self.base_params = {'user': self.username,'pass2': self.password, 'softid': self.soft_id,}
        self.headers = {'Connection': 'Keep-Alive',
            'User-Agent': 'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0)', }
    def PostPic(self, im, codetype):
        """im: 图片字节,codetype: 题目类型 参考 http://www.chaojiying.com/price.html"""
        params = { 'codetype': codetype,}
        params.update(self.base_params)
        files = {'userfile': ('ccc.jpg', im)}
        r = requests.post('http://upload.chaojiying.net/Upload/Processing.php', data=params, files=files, headers=self.headers)
        return r.json()
    def ReportError(self, im_id):
        """ im_id:报错题目的图片ID"""
        params = {'id': im_id,}
        params.update(self.base_params)
        r = requests.post('http://upload.chaojiying.net/Upload/ReportError.php', data=params, headers=self.headers)
        return r.json()

def submit_img():       # 将验证码提交给超级鹰进行识别
    chaojiying = Chaojiying_Client('18820147698', 'cfy122333..', '915194')  # 用户中心>>软件ID 生成一个
    im = open(r'C:\Users\hp\Desktop\small_img.png', 'rb').read()            # 本地图片文件路径 来替换 a.jpg 有时WIN系统须要//
    result = chaojiying.PostPic(im, 9004)['pic_str']
    print('调用超级鹰识别——》识别结果：图片中的验证码字符位置 ',result)
    return result

def parse_data(result):
    all_list = []       # 存储即将被点击的点的坐标  [[x1,y1],[x2,y2]]
    if '|' in result:
        list_1 = result.split('|')
        count_1 = len(list_1)
        for i in range(count_1):
            xy_list = []
            x = int(list_1[i].split(',')[0])
            y = int(list_1[i].split(',')[1])
            xy_list.append(x)
            xy_list.append(y)
            all_list.append(xy_list)
    else:
        print(result.split(',')[0])
        print(result.split(',')[1])

        x = int(result.split(',')[0])
        y = int(result.split(',')[1])
        xy_list = []
        xy_list.append(x)
        xy_list.append(y)
        all_list.append(xy_list)
    print(all_list)
    return all_list

def click_codeImg(all_list, code_img_ele):
    for l in all_list:       # 遍历列表，使用动作链对每一个列表元素对应的x,y指定的位置进行点击操作
        x = l[0]*1.6
        y = l[1]*1.6
        # move_to_element_with_offset移动到距某个元素（左上角坐标）多少距离的位置
        ActionChains(bro).move_to_element_with_offset(code_img_ele, x, y).click().perform()
        sleep(random.random()*1.5)
        print('点击已完成')
    sleep(random.random()*2)
    # 完成动作链点击操作后，定位确认按钮并点击
    bro.find_element_by_xpath('/html/body/div[3]/div[2]/div[6]/div/div/div[3]/a').click()

def main():
    login()                       # 进入登录界面，输入账号密码
    code_img_ele = save_img()     # 保存页面截图，并根据坐标裁剪获取验证码图片
    narrow_img()                  # 缩小图片
    result = submit_img()         # 将图片提交给超级鹰,获取返回值
    all_list = parse_data(result)             # 解析返回值,将数据格式化
    click_codeImg(all_list, code_img_ele)     # 在页面验证码上完成点击操作
main()

验证码位置： {'x': 792, 'y': 240} 验证码size： {'height': 346, 'width': 270}
调用超级鹰识别——》识别结果：图片中的验证码字符位置  41,69|101,33|40,102|120,86
[[41, 69], [101, 33], [40, 102], [120, 86]]
点击已完成
点击已完成
点击已完成
点击已完成
